In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import skipgrams
from tensorflow.keras.models import Sequential , Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input, Dot
from tensorflow.keras.utils import plot_model
import nltk
import gensim

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
f = open('/content/drive/MyDrive/KDT/9. 자연어 처리/modi_output.txt', 'r')
dataset = f.read()
dataset = dataset.split('\t')
f.close()

In [4]:
modi_df = pd.DataFrame({'document':dataset})
modi_df

,document
0,자신의 생각을 불명확하게 표현하는 사람들은 생각도 불명확할 가능성이 높다. 따라서 ...
1,주제가 아무리 복잡하고 어려울 지라도 필자가 주제에 대해 완벽하게 이해하고 있다면 ...
2,"진로체험을 위해서 교사 외에도 인솔, 안전 등의 일용직 인력이 필요하다. 재정과 고..."
3,Mary Footer는 최근에 WTO법의 모순을 해결하기 위해 다섯가지를 제안했다....
4,"에너지원별 소비 비중 중 전력비중은 지속적으로 증가하였고, 이것은 원자력 발전 설비..."
...,...
160019,"한정애안과 양창영안은 등록제를 제안하고, 장하나안과 유럽연합은 허가제·면허제를 도입..."
160020,토지이용의 변화로 탄소가 풍부한 유출물 침전과 지표유출수 내 고형물 퇴적은 정비하천...
160021,"NCATS는 이 외에도 여러 사업들을 진행하는데, 대부분이 공동연구이다. 약물 개발..."
160022,"국내는 평가기술에 대한 과학적 지식, 경제성 분석, 사회적 효과가 하나의 보고서 안..."


In [5]:
# 데이터셋에 결측값이 있는지 확인하기
# 빈것을 NAN으로 만들어 검사해보기
modi_df.replace('',float('NaN'), inplace=True)
print(modi_df.isnull().values.any())

False


In [6]:
# 열을 기준으로 중복된 데이터를 제거
processed_modi_df = modi_df.drop_duplicates(['document']).reset_index(drop=True)
processed_modi_df

,document
0,자신의 생각을 불명확하게 표현하는 사람들은 생각도 불명확할 가능성이 높다. 따라서 ...
1,주제가 아무리 복잡하고 어려울 지라도 필자가 주제에 대해 완벽하게 이해하고 있다면 ...
2,"진로체험을 위해서 교사 외에도 인솔, 안전 등의 일용직 인력이 필요하다. 재정과 고..."
3,Mary Footer는 최근에 WTO법의 모순을 해결하기 위해 다섯가지를 제안했다....
4,"에너지원별 소비 비중 중 전력비중은 지속적으로 증가하였고, 이것은 원자력 발전 설비..."
...,...
160019,"한정애안과 양창영안은 등록제를 제안하고, 장하나안과 유럽연합은 허가제·면허제를 도입..."
160020,토지이용의 변화로 탄소가 풍부한 유출물 침전과 지표유출수 내 고형물 퇴적은 정비하천...
160021,"NCATS는 이 외에도 여러 사업들을 진행하는데, 대부분이 공동연구이다. 약물 개발..."
160022,"국내는 평가기술에 대한 과학적 지식, 경제성 분석, 사회적 효과가 하나의 보고서 안..."


In [7]:
# 데이터셋에 특수 문자를 제거
processed_modi_df['document'] = processed_modi_df['document'].str.replace('[^가-힣]', ' ')
processed_modi_df

<ipython-input-7-65b848779a8d>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  processed_modi_df['document'] = processed_modi_df['document'].str.replace('[^가-힣]', ' ')


,document
0,자신의 생각을 불명확하게 표현하는 사람들은 생각도 불명확할 가능성이 높다 따라서 ...
1,주제가 아무리 복잡하고 어려울 지라도 필자가 주제에 대해 완벽하게 이해하고 있다면 ...
2,진로체험을 위해서 교사 외에도 인솔 안전 등의 일용직 인력이 필요하다 재정과 고...
3,는 최근에 법의 모순을 해결하기 위해 다섯가지를 제안했다 ...
4,에너지원별 소비 비중 중 전력비중은 지속적으로 증가하였고 이것은 원자력 발전 설비...
...,...
160019,한정애안과 양창영안은 등록제를 제안하고 장하나안과 유럽연합은 허가제 면허제를 도입...
160020,토지이용의 변화로 탄소가 풍부한 유출물 침전과 지표유출수 내 고형물 퇴적은 정비하천...
160021,는 이 외에도 여러 사업들을 진행하는데 대부분이 공동연구이다 약물 개발...
160022,국내는 평가기술에 대한 과학적 지식 경제성 분석 사회적 효과가 하나의 보고서 안...


In [8]:
# 전체 길이가 200 이하이거나 전체 단어 개수가 5개 이하인 데이터를 필터링
processed_modi_df = processed_modi_df[processed_modi_df.document.apply(lambda x: len(str(x)) > 200 and len(str(x).split()) > 5)].reset_index(drop=True)
processed_modi_df

,document
0,자신의 생각을 불명확하게 표현하는 사람들은 생각도 불명확할 가능성이 높다 따라서 ...
1,주제가 아무리 복잡하고 어려울 지라도 필자가 주제에 대해 완벽하게 이해하고 있다면 ...
2,진로체험을 위해서 교사 외에도 인솔 안전 등의 일용직 인력이 필요하다 재정과 고...
3,는 최근에 법의 모순을 해결하기 위해 다섯가지를 제안했다 ...
4,에너지원별 소비 비중 중 전력비중은 지속적으로 증가하였고 이것은 원자력 발전 설비...
...,...
159998,위험요인을 보완하지 않은 상황에서 라식 수술에 표면절제술이 사용되었다면 각막혼탁이 ...
159999,한정애안과 양창영안은 등록제를 제안하고 장하나안과 유럽연합은 허가제 면허제를 도입...
160000,토지이용의 변화로 탄소가 풍부한 유출물 침전과 지표유출수 내 고형물 퇴적은 정비하천...
160001,는 이 외에도 여러 사업들을 진행하는데 대부분이 공동연구이다 약물 개발...


In [9]:
import requests

In [10]:
url = 'https://raw.githubusercontent.com/stopwords-iso/stopwords-ko/master/raw/ranksnl-korean.txt'
response = requests.get(url)
content = response.text
stop_words = content.split('\n')
print(len(stop_words))
print(stop_words[:10])

677
['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라']


In [11]:
tokenized_doc = processed_modi_df['document'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [s_word for s_word in x if s_word not in stop_words])
tokenized_doc

0         [자신의, 생각을, 불명확하게, 표현하는, 사람들은, 생각도, 불명확할, 가능성이,...
1         [주제가, 아무리, 복잡하고, 어려울, 지라도, 필자가, 주제에, 대해, 완벽하게,...
2         [진로체험을, 교사, 인솔, 안전, 등의, 일용직, 인력이, 필요하다, 재정과, 고...
3         [는, 최근에, 법의, 모순을, 해결하기, 위해, 다섯가지를, 제안했다, 그녀가, ...
4         [에너지원별, 소비, 비중, 중, 전력비중은, 지속적으로, 증가하였고, 이것은, 원...
                                ...                        
159998    [위험요인을, 보완하지, 않은, 상황에서, 라식, 수술에, 표면절제술이, 사용되었다...
159999    [한정애안과, 양창영안은, 등록제를, 제안하고, 장하나안과, 유럽연합은, 허가제, ...
160000    [토지이용의, 변화로, 탄소가, 풍부한, 유출물, 침전과, 지표유출수, 내, 고형물...
160001    [는, 여러, 사업들을, 진행하는데, 대부분이, 공동연구이다, 약물, 개발, 관련,...
160002    [국내는, 평가기술에, 대한, 과학적, 지식, 경제성, 분석, 사회적, 효과가, 하...
Name: document, Length: 160003, dtype: object

In [12]:
tokenized_doc = tokenized_doc.to_list()
print(len(tokenized_doc))

160003


In [13]:
tokenizer = Tokenizer()

In [14]:
tokenizer.fit_on_texts(tokenized_doc)

In [15]:
word2idx = tokenizer.word_index
idx2word = {value: key for key, value in word2idx.items()}
encoded = tokenizer.texts_to_sequences(tokenized_doc)

In [16]:
vocab_size = len(word2idx) + 1
print(f'단어 사전의 크기: {vocab_size}')

단어 사전의 크기: 1332492


In [17]:
skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded[:5]]
print(f'전체 샘플 수: {len(skip_grams)}')

전체 샘플 수: 5


In [18]:
# skip_grams[0]에 skipgrams로 형성된 데이터셋 확인
pairs, labels = skip_grams[0][0], skip_grams[0][1]
print(f'3 pairs: {pairs[:3]}')
print(f'3 labels: {labels[:3]}')

3 pairs: [[162943, 409], [596, 282], [57, 208143]]
3 labels: [1, 1, 1]


In [19]:
print(len(pairs))
print(len(labels))

3500
3500


In [20]:
for i in range(5):
    print('({:s} ({:d}), {:s} ({:d})) -> {:d}'.format(
        idx2word[pairs[i][0]], pairs[i][0],
        idx2word[pairs[i][1]], pairs[i][1],
        labels[i]
    ))

(케네스 (162943), 가능하다 (409)) -> 1
(사용하는 (596), 중요하다 (282)) -> 1
(된다 (57), 쓰려면 (208143)) -> 1
(생각도 (27838), 민사범죄에는 (938582)) -> 0
(어려운 (245), 병리현상같은 (808389)) -> 0


In [21]:
training_dataset = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded[:5000]]

In [22]:
embedding_dim = 100

w_inputs = Input(shape=(1,), dtype='int32')
word_embedding = Embedding(vocab_size, embedding_dim)(w_inputs)

c_inputs = Input(shape=(1,), dtype='int32')
context_embedding = Embedding(vocab_size, embedding_dim)(c_inputs)

In [23]:
dot_product = Dot(axes=2)([word_embedding, context_embedding])
dot_product = Reshape((1,), input_shape=(1, 1))(dot_product)
output = Activation('sigmoid')(dot_product)

model = Model(inputs = [w_inputs, c_inputs], outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 100)       133249200   ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 1, 100)       133249200   ['input_2[0][0]']                
                                                                                              

In [24]:
model.compile(loss='binary_crossentropy', optimizer='adam')

In [25]:
for epoch in range(100):
    loss = 0
    for _, elem in enumerate(skip_grams):
        first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
        second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
        labels = np.array(elem[1], dtype='int32')
        X = [first_elem, second_elem]
        Y = labels
        loss += model.train_on_batch(X, Y)
    print('Epoch: ', epoch+1, 'Loss: ', loss)

Epoch:  1 Loss:  3.4655929803848267
Epoch:  2 Loss:  3.458336293697357
Epoch:  3 Loss:  3.4507569670677185
Epoch:  4 Loss:  3.442359507083893
Epoch:  5 Loss:  3.4328315258026123
Epoch:  6 Loss:  3.4218326210975647
Epoch:  7 Loss:  3.409018874168396
Epoch:  8 Loss:  3.394054114818573
Epoch:  9 Loss:  3.376616954803467
Epoch:  10 Loss:  3.3564049005508423
Epoch:  11 Loss:  3.3331382870674133
Epoch:  12 Loss:  3.3065654635429382
Epoch:  13 Loss:  3.276466727256775
Epoch:  14 Loss:  3.242661952972412
Epoch:  15 Loss:  3.205010771751404
Epoch:  16 Loss:  3.163420081138611
Epoch:  17 Loss:  3.11784428358078
Epoch:  18 Loss:  3.0682893991470337
Epoch:  19 Loss:  3.0148106813430786
Epoch:  20 Loss:  2.9575151205062866
Epoch:  21 Loss:  2.896557867527008
Epoch:  22 Loss:  2.832141876220703
Epoch:  23 Loss:  2.7645119428634644
Epoch:  24 Loss:  2.693952739238739
Epoch:  25 Loss:  2.620782971382141
Epoch:  26 Loss:  2.5453493297100067
Epoch:  27 Loss:  2.468020260334015
Epoch:  28 Loss:  2.389180

In [26]:
f = open('vectors.txt', 'w')
f.write('{} {}\n'.format(vocab_size-1, embedding_dim))
vectors = model.get_weights()[0]
# print(vectors)
for word, i in tokenizer.word_index.items():
    f.write('{} {}\n'.format(word, ' '.join(map(str, list(vectors[i, :])))))
f.close()

In [27]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)

In [35]:
w2v.most_similar(positive=['사전'])

[('건재', 0.4593060314655304),
 ('재활의', 0.4430803954601288),
 ('법원이던', 0.437846839427948),
 ('여신관련', 0.42684653401374817),
 ('기업경영자로서', 0.4267728924751282),
 ('의탁하는', 0.4254499673843384),
 ('교원노동조합은', 0.42202049493789673),
 ('개월간씩', 0.42176374793052673),
 ('맞벌이부부에게', 0.4171050488948822),
 ('식별하기가', 0.4125789403915405)]